In [ ]:
SELECT *
FROM flight f
WHERE NOT EXISTS
	( 
		SELECT * 
	 	FROM airline_city ac 
		WHERE f.destination = ac.code and f.airlineid = ac.airlineid
	)
;

In [ ]:
import psycopg2
import pandas as pd

params_dic = {
    "host"      : "localhost",
    "dbname"    : "flights",
    "user"      : "postgres",
    "password"  : "password",
    "port" : "..."     
}

conn = psycopg2.connect(**params_dic)

cursor = conn.cursor()
cursor.execute("SELECT * FROM airline_city")
rows = cursor.fetchall()
print(rows)

cursor.close()

df = pd.DataFrame(rows, columns=["airlineid", "ap_name", "code"])
df.head()

comprehensive explanation of code below:

In [ ]:
import psycopg2
import pandas as pd

# set up log in information
params_dic = {
    "host"      : "localhost",
    "dbname"    : "flights",
    "user"      : "postgres",
    "password"  : "password",
    "port" : "..."      # this is the port set up by your default wizard
}
# we can use "SELECT * FROM pg_settings WHERE name = 'port';" in pgAdmin to discover our port number
# learn more about ports here: https://www.cloudflare.com/learning/network-layer/what-is-a-computer-port/

# connect to your database using the dictionary above
# the ** operator unpacks all your settings into their appropriate params
# without the ** operator, we would have to manually set these params via
# psycopg2.connect(host="localhost", dbname="flights", user="postgres", password="password", port="...")
conn = psycopg2.connect(**params_dic)

# create a cursor
# think of a cursor like an object that stores and executes queries
# we prepare it by calling conn.cursor() in Python
# for our purpose, a cursor is an object that allows us to execute queries 
# learn more about cursors here: https://www.geeksforgeeks.org/what-is-cursor-in-sql/
cursor = conn.cursor()

# execute a query using the cursor
# docs of cursor here: https://www.psycopg.org/docs/cursor.html
cursor.execute("SELECT * FROM airline_city")

# pull all rows from the query you just executed
rows = cursor.fetchall()
print(rows)

# close your cursor, must be done once you are done interacting w/your cursor
cursor.close()

# save your list of tuples (representing rows) into a dataframe!
df = pd.DataFrame(rows, columns=["airlineid", "ap_name", "code"])

# print out the first 5 rows of your dataframe!
df.head()